In [1]:
import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64

from matplotlib.path import Path
import matplotlib.patches as patches

In [2]:
def strokes_to_lines(strokes):
    """
    Convert stroke-3 format to polyline format.
    List contains sublist of continuous line segments (strokes).    
    """
    x = 0
    y = 0
    lines = []
    line = []
    for i in range(len(strokes)):
        if strokes[i, 2] == 1:
            x += float(strokes[i, 0])
            y += float(strokes[i, 1])
            line.append([x, y])
            lines.append(line)
            line = []
        else:
            x += float(strokes[i, 0])
            y += float(strokes[i, 1])
            line.append([x, y])
    return lines

def polyline_pathmaker(lines):
    x = []
    y = []

    codes = [Path.MOVETO] # start with moveto command always
    for i,l in enumerate(lines):
        for _i,_l in enumerate(l):
            x.append(_l[0])
            y.append(_l[1])
            if _i<len(l)-1:
                codes.append(Path.LINETO) # keep pen on page
            else:
                if i != len(lines)-1: # final vertex
                    codes.append(Path.MOVETO)
    verts = zip(x,y)            
    return verts, codes

def path_renderer(verts, codes, save_path):
    fig = plt.figure(figsize=(6,6))    
    ax = plt.subplot(111)    
    padding = 10
    xlb = min([x for (x,y) in verts]) - padding
    xub = max([x for (x,y) in verts]) + padding
    ylb = min([y for (x,y) in verts]) - padding
    yub = max([y for (x,y) in verts]) + padding
    lb = min(xlb,ylb)
    ub = max(xub,yub)
    ax.axis('off')
    
    ax.set_xlim(lb,ub)
    ax.set_ylim(lb,ub)     
    
    if len(verts)>0:
        path = Path(verts, codes)
        patch = patches.PathPatch(path, facecolor='none', lw=5)
        ax.add_patch(patch)
        plt.gca().invert_yaxis() # y values increase as you go down in image
#         plt.show() 
    
    if len(save_path)>0:
        print save_path
        fig.savefig(save_path,bbox_inches='tight')
        # load, resize, then resave
        im = Image.open(save_path)
        im = im.resize((224,224), Image.ANTIALIAS)
        im.save(save_path)          

In [3]:
# path to npz files
path_npz = '/data2/jefan/quickDraw/npz'
path_png = '/data2/jefan/quickDraw/png'

# how many drawings from each category to include
sample_size = 100

In [4]:
# sorted list of all categories
paths = [os.path.join(path_npz,i) for i in sorted(os.listdir(path_npz))]

In [ ]:
# loop through paths and render out
for path in paths:
    if path.split('/')[-1] != 'ignore':
        print path
        category = path.split('F')[1].split('.')[0]
        x = np.load(path)
        y = x['train']
        for i in np.arange(sample_size):    
            lines = strokes_to_lines(y[i])
            verts,codes = polyline_pathmaker(lines)
            save_path = os.path.join(path_png,category,'{:04d}.png'.format(i))   
            if not os.path.exists(os.path.join(path_png,category)):
                os.makedirs(os.path.join(path_png,category))
            path_renderer(verts, codes,save_path)
            plt.close('all')

### Jan 14 2018: render out smaller number of adult sketches from fewer categories (only the ones for which we have sufficient kid data

In [5]:
path_npz_mini = '/data2/jefan/quickDraw/npz'
path_png_mini = '/data2/jefan/quickDraw/png_mini'

# the categories for which we have enough kid data
sufficient_labels = ['banana',
                     'bird',
                     'boat',
                     'car',
                     'carrot',
                     'chair',
                     'couch',
                     'cup',
                     'flower',
                     'foot',
                     'frog',
                     'ice cream',
                     'phone',
                     'rabbit',
                     'shoe',
                     'train']

In [6]:
# sorted list of all categories
paths = [os.path.join(path_npz_mini,i) for i in sorted(os.listdir(path_npz_mini)) if i != 'ignore']

# how many drawings from each category to include
sample_size = 500

In [ ]:
# loop through paths and render out
for path in paths:
    print path
    category = path.split('F')[1].split('.')[0]
    x = np.load(path)
    y = x['train']
    for i in np.arange(sample_size):    
        lines = strokes_to_lines(y[i])
        verts,codes = polyline_pathmaker(lines)
        save_path = os.path.join(path_png_mini,category,'{:04d}.png'.format(i))   
        if not os.path.exists(os.path.join(path_png_mini,category)):
            os.makedirs(os.path.join(path_png_mini,category))
        path_renderer(verts, codes,save_path)
        plt.close('all')


/data2/jefan/quickDraw/npz/sketchrnn%2Fairplane.npz
/data2/jefan/quickDraw/png_mini/airplane/0000.png
/data2/jefan/quickDraw/png_mini/airplane/0001.png
/data2/jefan/quickDraw/png_mini/airplane/0002.png
/data2/jefan/quickDraw/png_mini/airplane/0003.png
/data2/jefan/quickDraw/png_mini/airplane/0004.png
/data2/jefan/quickDraw/png_mini/airplane/0005.png
/data2/jefan/quickDraw/png_mini/airplane/0006.png
/data2/jefan/quickDraw/png_mini/airplane/0007.png
/data2/jefan/quickDraw/png_mini/airplane/0008.png
/data2/jefan/quickDraw/png_mini/airplane/0009.png
/data2/jefan/quickDraw/png_mini/airplane/0010.png
/data2/jefan/quickDraw/png_mini/airplane/0011.png
/data2/jefan/quickDraw/png_mini/airplane/0012.png
/data2/jefan/quickDraw/png_mini/airplane/0013.png
/data2/jefan/quickDraw/png_mini/airplane/0014.png
/data2/jefan/quickDraw/png_mini/airplane/0015.png
/data2/jefan/quickDraw/png_mini/airplane/0016.png
/data2/jefan/quickDraw/png_mini/airplane/0017.png
/data2/jefan/quickDraw/png_mini/airplane/0018.pn

/data2/jefan/quickDraw/png_mini/airplane/0163.png
/data2/jefan/quickDraw/png_mini/airplane/0164.png
/data2/jefan/quickDraw/png_mini/airplane/0165.png
/data2/jefan/quickDraw/png_mini/airplane/0166.png
/data2/jefan/quickDraw/png_mini/airplane/0167.png
/data2/jefan/quickDraw/png_mini/airplane/0168.png
/data2/jefan/quickDraw/png_mini/airplane/0169.png
/data2/jefan/quickDraw/png_mini/airplane/0170.png
/data2/jefan/quickDraw/png_mini/airplane/0171.png
/data2/jefan/quickDraw/png_mini/airplane/0172.png
/data2/jefan/quickDraw/png_mini/airplane/0173.png
/data2/jefan/quickDraw/png_mini/airplane/0174.png
/data2/jefan/quickDraw/png_mini/airplane/0175.png
/data2/jefan/quickDraw/png_mini/airplane/0176.png
/data2/jefan/quickDraw/png_mini/airplane/0177.png
/data2/jefan/quickDraw/png_mini/airplane/0178.png
/data2/jefan/quickDraw/png_mini/airplane/0179.png
/data2/jefan/quickDraw/png_mini/airplane/0180.png
/data2/jefan/quickDraw/png_mini/airplane/0181.png
/data2/jefan/quickDraw/png_mini/airplane/0182.png


/data2/jefan/quickDraw/png_mini/airplane/0327.png
/data2/jefan/quickDraw/png_mini/airplane/0328.png
/data2/jefan/quickDraw/png_mini/airplane/0329.png
/data2/jefan/quickDraw/png_mini/airplane/0330.png
/data2/jefan/quickDraw/png_mini/airplane/0331.png
/data2/jefan/quickDraw/png_mini/airplane/0332.png
/data2/jefan/quickDraw/png_mini/airplane/0333.png
/data2/jefan/quickDraw/png_mini/airplane/0334.png
/data2/jefan/quickDraw/png_mini/airplane/0335.png
/data2/jefan/quickDraw/png_mini/airplane/0336.png
/data2/jefan/quickDraw/png_mini/airplane/0337.png
/data2/jefan/quickDraw/png_mini/airplane/0338.png
/data2/jefan/quickDraw/png_mini/airplane/0339.png
/data2/jefan/quickDraw/png_mini/airplane/0340.png
/data2/jefan/quickDraw/png_mini/airplane/0341.png
/data2/jefan/quickDraw/png_mini/airplane/0342.png
/data2/jefan/quickDraw/png_mini/airplane/0343.png
/data2/jefan/quickDraw/png_mini/airplane/0344.png
/data2/jefan/quickDraw/png_mini/airplane/0345.png
/data2/jefan/quickDraw/png_mini/airplane/0346.png


/data2/jefan/quickDraw/png_mini/airplane/0491.png
/data2/jefan/quickDraw/png_mini/airplane/0492.png
/data2/jefan/quickDraw/png_mini/airplane/0493.png
/data2/jefan/quickDraw/png_mini/airplane/0494.png
/data2/jefan/quickDraw/png_mini/airplane/0495.png
/data2/jefan/quickDraw/png_mini/airplane/0496.png
/data2/jefan/quickDraw/png_mini/airplane/0497.png
/data2/jefan/quickDraw/png_mini/airplane/0498.png
/data2/jefan/quickDraw/png_mini/airplane/0499.png
/data2/jefan/quickDraw/npz/sketchrnn%2Fapple.npz
/data2/jefan/quickDraw/png_mini/apple/0000.png
/data2/jefan/quickDraw/png_mini/apple/0001.png
/data2/jefan/quickDraw/png_mini/apple/0002.png
/data2/jefan/quickDraw/png_mini/apple/0003.png
/data2/jefan/quickDraw/png_mini/apple/0004.png
/data2/jefan/quickDraw/png_mini/apple/0005.png
/data2/jefan/quickDraw/png_mini/apple/0006.png
/data2/jefan/quickDraw/png_mini/apple/0007.png
/data2/jefan/quickDraw/png_mini/apple/0008.png
/data2/jefan/quickDraw/png_mini/apple/0009.png
/data2/jefan/quickDraw/png_mini

/data2/jefan/quickDraw/png_mini/apple/0165.png
/data2/jefan/quickDraw/png_mini/apple/0166.png
/data2/jefan/quickDraw/png_mini/apple/0167.png
/data2/jefan/quickDraw/png_mini/apple/0168.png
/data2/jefan/quickDraw/png_mini/apple/0169.png
/data2/jefan/quickDraw/png_mini/apple/0170.png
/data2/jefan/quickDraw/png_mini/apple/0171.png
/data2/jefan/quickDraw/png_mini/apple/0172.png
/data2/jefan/quickDraw/png_mini/apple/0173.png
/data2/jefan/quickDraw/png_mini/apple/0174.png
/data2/jefan/quickDraw/png_mini/apple/0175.png
/data2/jefan/quickDraw/png_mini/apple/0176.png
/data2/jefan/quickDraw/png_mini/apple/0177.png
/data2/jefan/quickDraw/png_mini/apple/0178.png
/data2/jefan/quickDraw/png_mini/apple/0179.png
/data2/jefan/quickDraw/png_mini/apple/0180.png
/data2/jefan/quickDraw/png_mini/apple/0181.png
/data2/jefan/quickDraw/png_mini/apple/0182.png
/data2/jefan/quickDraw/png_mini/apple/0183.png
/data2/jefan/quickDraw/png_mini/apple/0184.png
/data2/jefan/quickDraw/png_mini/apple/0185.png
/data2/jefan/

/data2/jefan/quickDraw/png_mini/apple/0341.png
/data2/jefan/quickDraw/png_mini/apple/0342.png
/data2/jefan/quickDraw/png_mini/apple/0343.png
/data2/jefan/quickDraw/png_mini/apple/0344.png
/data2/jefan/quickDraw/png_mini/apple/0345.png
/data2/jefan/quickDraw/png_mini/apple/0346.png
/data2/jefan/quickDraw/png_mini/apple/0347.png
/data2/jefan/quickDraw/png_mini/apple/0348.png
/data2/jefan/quickDraw/png_mini/apple/0349.png
/data2/jefan/quickDraw/png_mini/apple/0350.png
/data2/jefan/quickDraw/png_mini/apple/0351.png
/data2/jefan/quickDraw/png_mini/apple/0352.png
/data2/jefan/quickDraw/png_mini/apple/0353.png
/data2/jefan/quickDraw/png_mini/apple/0354.png
/data2/jefan/quickDraw/png_mini/apple/0355.png
/data2/jefan/quickDraw/png_mini/apple/0356.png
/data2/jefan/quickDraw/png_mini/apple/0357.png
/data2/jefan/quickDraw/png_mini/apple/0358.png
/data2/jefan/quickDraw/png_mini/apple/0359.png
/data2/jefan/quickDraw/png_mini/apple/0360.png
/data2/jefan/quickDraw/png_mini/apple/0361.png
/data2/jefan/

/data2/jefan/quickDraw/png_mini/banana/0015.png
/data2/jefan/quickDraw/png_mini/banana/0016.png
/data2/jefan/quickDraw/png_mini/banana/0017.png
/data2/jefan/quickDraw/png_mini/banana/0018.png
/data2/jefan/quickDraw/png_mini/banana/0019.png
/data2/jefan/quickDraw/png_mini/banana/0020.png
/data2/jefan/quickDraw/png_mini/banana/0021.png
/data2/jefan/quickDraw/png_mini/banana/0022.png
/data2/jefan/quickDraw/png_mini/banana/0023.png
/data2/jefan/quickDraw/png_mini/banana/0024.png
/data2/jefan/quickDraw/png_mini/banana/0025.png
/data2/jefan/quickDraw/png_mini/banana/0026.png
/data2/jefan/quickDraw/png_mini/banana/0027.png
/data2/jefan/quickDraw/png_mini/banana/0028.png
/data2/jefan/quickDraw/png_mini/banana/0029.png
/data2/jefan/quickDraw/png_mini/banana/0030.png
/data2/jefan/quickDraw/png_mini/banana/0031.png
/data2/jefan/quickDraw/png_mini/banana/0032.png
/data2/jefan/quickDraw/png_mini/banana/0033.png
/data2/jefan/quickDraw/png_mini/banana/0034.png
/data2/jefan/quickDraw/png_mini/banana/0

/data2/jefan/quickDraw/png_mini/banana/0187.png
/data2/jefan/quickDraw/png_mini/banana/0188.png
/data2/jefan/quickDraw/png_mini/banana/0189.png
/data2/jefan/quickDraw/png_mini/banana/0190.png
/data2/jefan/quickDraw/png_mini/banana/0191.png
/data2/jefan/quickDraw/png_mini/banana/0192.png
/data2/jefan/quickDraw/png_mini/banana/0193.png
/data2/jefan/quickDraw/png_mini/banana/0194.png
/data2/jefan/quickDraw/png_mini/banana/0195.png
/data2/jefan/quickDraw/png_mini/banana/0196.png
/data2/jefan/quickDraw/png_mini/banana/0197.png
/data2/jefan/quickDraw/png_mini/banana/0198.png
/data2/jefan/quickDraw/png_mini/banana/0199.png
/data2/jefan/quickDraw/png_mini/banana/0200.png
/data2/jefan/quickDraw/png_mini/banana/0201.png
/data2/jefan/quickDraw/png_mini/banana/0202.png
/data2/jefan/quickDraw/png_mini/banana/0203.png
/data2/jefan/quickDraw/png_mini/banana/0204.png
/data2/jefan/quickDraw/png_mini/banana/0205.png
/data2/jefan/quickDraw/png_mini/banana/0206.png
/data2/jefan/quickDraw/png_mini/banana/0

/data2/jefan/quickDraw/png_mini/banana/0359.png
/data2/jefan/quickDraw/png_mini/banana/0360.png
/data2/jefan/quickDraw/png_mini/banana/0361.png
/data2/jefan/quickDraw/png_mini/banana/0362.png
/data2/jefan/quickDraw/png_mini/banana/0363.png
/data2/jefan/quickDraw/png_mini/banana/0364.png
/data2/jefan/quickDraw/png_mini/banana/0365.png
/data2/jefan/quickDraw/png_mini/banana/0366.png
/data2/jefan/quickDraw/png_mini/banana/0367.png
/data2/jefan/quickDraw/png_mini/banana/0368.png
/data2/jefan/quickDraw/png_mini/banana/0369.png
/data2/jefan/quickDraw/png_mini/banana/0370.png
/data2/jefan/quickDraw/png_mini/banana/0371.png
/data2/jefan/quickDraw/png_mini/banana/0372.png
/data2/jefan/quickDraw/png_mini/banana/0373.png
/data2/jefan/quickDraw/png_mini/banana/0374.png
/data2/jefan/quickDraw/png_mini/banana/0375.png
/data2/jefan/quickDraw/png_mini/banana/0376.png
/data2/jefan/quickDraw/png_mini/banana/0377.png
/data2/jefan/quickDraw/png_mini/banana/0378.png
/data2/jefan/quickDraw/png_mini/banana/0

/data2/jefan/quickDraw/png_mini/bear/0030.png
/data2/jefan/quickDraw/png_mini/bear/0031.png
/data2/jefan/quickDraw/png_mini/bear/0032.png
/data2/jefan/quickDraw/png_mini/bear/0033.png
/data2/jefan/quickDraw/png_mini/bear/0034.png
/data2/jefan/quickDraw/png_mini/bear/0035.png
/data2/jefan/quickDraw/png_mini/bear/0036.png
/data2/jefan/quickDraw/png_mini/bear/0037.png
/data2/jefan/quickDraw/png_mini/bear/0038.png
/data2/jefan/quickDraw/png_mini/bear/0039.png
/data2/jefan/quickDraw/png_mini/bear/0040.png
/data2/jefan/quickDraw/png_mini/bear/0041.png
/data2/jefan/quickDraw/png_mini/bear/0042.png
/data2/jefan/quickDraw/png_mini/bear/0043.png
/data2/jefan/quickDraw/png_mini/bear/0044.png
/data2/jefan/quickDraw/png_mini/bear/0045.png
/data2/jefan/quickDraw/png_mini/bear/0046.png
/data2/jefan/quickDraw/png_mini/bear/0047.png
/data2/jefan/quickDraw/png_mini/bear/0048.png
/data2/jefan/quickDraw/png_mini/bear/0049.png
/data2/jefan/quickDraw/png_mini/bear/0050.png
/data2/jefan/quickDraw/png_mini/be